# 1. Package Installation

In [20]:
#@title Requirements
%%writefile requirements.txt
peft
fire
accelerator
transformers
datasets
evaluate
pyarrow
galore-torch
pytorch-ignite
rouge-score
nltk
py7zr
optimum[exporters]
trl

Overwriting requirements.txt


In [21]:
#@title Install Packages
%%capture
!pip install --no-cache -r requirements.txt

In [3]:
#@title Huggingface Login
#@markdown huggingface weight 를 이용하고 싶다면 로그인 필수
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in 

# 2. Load Model


In [73]:
#@title Get peft model from huggingface
#@markdown Colab 고용량 Ram CPU에서 가능한 범위 ~8B(테스트 중)
%%writefile peft_model.py

import os
import fire
import torch
from peft import AutoPeftModelForCausalLM
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from random import randint

base_model_id = "google/gemma-7b" # @param ["meta-llama/Meta-Llama-3-8B", "meta-llama/Meta-Llama-3-70B-Instruct", "mistralai/Mistral-7B-Instruct-v0.3", "Qwen/Qwen2-7B-Instruct", "google/gemma-7b", "MLP-KTLim/llama-3-Korean-Bllossom-8B"] {allow-input: true}

peft_model = AutoModelForCausalLM.from_pretrained(base_model_id)

# adapter configuration
lora_config = LoraConfig(
    target_modules=["q_proj", "k_proj"],
    init_lora_weights="gaussian", #"gaussian", "pissa", "pissa_niter_{n}", "loftq", False
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    inference_mode=False,
    use_dora=False,
)

peft_model.add_adapter(lora_config, adapter_name="adapter_1")

tokenizer = AutoTokenizer.from_pretrained(base_model_id)


Writing peft_model.py


#3. Load Dataset

In [77]:
#@title Load data From huggingface datasets
#@markdown summary task에 대해 우선적으로 실험
%%writefile finetuning_datasets.py
from datasets import load_dataset
from evaluate import load

dataset_path = "Samsung/samsum" # @param ["Samsung/samsum", "emozilla/soda_synthetic_dialogue", "frcp/summary-alpaca-v01"] {allow-input: true}

dataset = load_dataset(
  dataset_path,
  revision="main"  # tag name, or branch name, or commit hash
)
metric = load("rouge")

Writing finetuning_datasets.py


#4. Train

In [78]:
#@title Start Training
#@markdown transformers trainer 이용, 추후 lightning 으로 이전 가능
%%writefile train.py
import nltk
import numpy as np
from torch.utils.data import DataLoader
from transformers import TrainingArguments, Trainer, TrainerCallback
from transformers import DataCollatorForSeq2Seq
from ignite.metrics import Rouge
from peft_model import peft_model, tokenizer
from finetuning_datasets import dataset, metric


# Callback Class
class EarlyStoppingCallback(TrainerCallback):
    def __init__(self, num_steps=10):
        self.num_steps = num_steps

    def on_step_end(self, args, state, control, **kwargs):
        if state.global_step >= self.num_steps:
            return {"should_training_stop": True}
        else:
            return {}

# metric function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

data_collator = DataCollatorForSeq2Seq(tokenizer, model=peft_model)

training_args = TrainingArguments(
    output_dir="llm_output",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)


trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback()],
)


trainer.train()


Overwriting train.py


In [ ]:
!python train.py

# Model Saving

In [ ]:
#@title ONNX model save
#@markdown ONNX 로 모델 변형 후 저장
from optimum.onnxruntime import ORTModelForSequenceClassification, ORTModelForCausalLM

model_checkpoint = "./" #@param{"type":"string"}
save_directory = "./" #@param{"type":"string"}

ort_model = ORTModelForCausalLM.from_pretrained(model_checkpoint, export=True)
ort_model.save_pretrained(save_directory)